In [7]:
import sys, ast, os
import networkx as nx
from PathEnumeration import activeBinPathEnumeration3, getMultiBins
from PostFilterPairedBin import groupPairedBins, eliminateInvalidPaths
from PairedBinsToBins_short import fromPairedBinsToBinsShort
from parse_graph_list_commented_Arbeitsdatei import parse_meta, parse_bins, parse_pairs, parse_graph, write_valid_gtf_entry, nodepath_to_transcript
from copy import deepcopy
from linearProgramming import addCounts, optimizeTranscripts, createAlphaIJMatrix, createAlphaIJDict
import numpy as np
import gurobipy as gp
from gurobipy import *
import math

#from printGraph import writeGeneToGraph

dummyf = open("dummyout.gtf", "w")                                                                                              # output for the dummy code
dummyGeneCounter = 0

with open('test3.graph') as f:
    fileEndReached = False
    f.readline()                                                                                                    #skip ---- seperator line
    
    # Define necessary Dictionaries                                                                                                     
    paths = {}
    filteredPaths = {}
    pairedBinDict = {}
    exonSpliceJunctionCountDict = {}

    #Define necessary lists
    pfadListe = []

    # Define necessary Integer Varibales
    geneCounter = 0
    enumerationPathCounter = 0
    validPathCounter = 0
    
    while not fileEndReached:
        f.readline()                                                                                                # skip ==META: Read this line, but don't do anything
        Chromosome, Strand, Exons = parse_meta(f)                                                                   # Übergib f jetzt an def parse_meta, um Metadaten auszulesen und schreib diese in Chromosome, 
                                                                                                                    # Strand und Exons (Listen)
        Bins = parse_bins(f)                                                                                        # Lies die BINS aus f mit Hilfe der parse_bin(f) Funktion aus und schreib sie in Bins
        PairedBins = parse_pairs(f)
        
        G_full = nx.DiGraph()                                                                                       # Erzeug einen Diagraphen mit Hilfe von NetworkX
        fileEndReached, skip = parse_graph(f, G_full, Exons)                                                        # Ruf die Funktion parse_graph auf, übergibt ihr das File (f, den erzeugten Graphen und zugehörige Liste 
                                                                                                                    # mit den Exons), schreib den vollen Graphen in G_full, weise Skip einen Boolean zu, der true ist, wenn 
                                                                                                                    # die Zeile mit - beginnt                                                                                                                                                                                                      
        if not fileEndReached and not skip:                                                                         # Falls denoised Graph existiert, ruf wieder die Funktion Parse_Graph auf, übergib ihr das File 
                                                                                                                    # (f, Graph_clean und zugehörige Liste mit den Exons), schreib die # denoised Informationen aus dem 
                                                                                                                    # Graph-File in graph clean und übergib die letzte Zeile fileEndReached
            G_clean = nx.DiGraph()                                                                                  # Erzeug einen neuen gerichteten Graphen 
            fileEndReached, _ = parse_graph(f, G_clean, Exons)
        
        else:
            G_clean = G_full

        # Define necessary variables and assign
            # pathNumbers to enumerate paths  
        
        enumerationPathNumber = [0]
        validPathNumber = [0]
        pairedBinsCopy = deepcopy(PairedBins)

        # # 1. Get MultiBins only from Bins
        MultiBins = getMultiBins(Bins)

        # # 3. Add PairedBins to MultiBins
        MultiBins, gepaarteBins = fromPairedBinsToBinsShort(PairedBins, MultiBins, G_clean, Exons, True)
        
        # # 4. Enumerate Paths with ActiveBinPathEnumeration3
        paths['Gene' + str(geneCounter)] = activeBinPathEnumeration3('1', '0', ['0'], {}, enumerationPathNumber, [], G_clean, MultiBins)

        # # 4a Enumerate paths with ActiveBinPathEnumeration3 (Bins)
        # #paths['Gene' + str(geneCounter)] = activeBinPathEnumeration3('1', '0', ['0'], {}, enumerationPathNumber, [], G_clean, MultiBins)

        # # 5. Group paired Bins to filter valid paths after Enumeration 
        groupedPairedBins = groupPairedBins(pairedBinsCopy)
        
        # # 6. Filter invalid Paths
        filteredPaths['Gene' + str(geneCounter)] = eliminateInvalidPaths(paths['Gene' + str(geneCounter)], groupedPairedBins, validPathNumber)

        # 7. Count paths
        enumerationPathCounter = enumerationPathCounter + enumerationPathNumber[0]
        validPathCounter = validPathCounter + validPathNumber[0]
        
        # 8. Extract Exon counts
        exonSpliceJunctionCountDict['Gene' + str(geneCounter)] = addCounts(G_clean)

        # Create np-Array
        alphaNP = createAlphaIJMatrix(exonSpliceJunctionCountDict['Gene' + str(geneCounter)], filteredPaths['Gene' + str(geneCounter)])

        #Create Dictionary
        alphaMultiDict = createAlphaIJDict(exonSpliceJunctionCountDict['Gene' + str(geneCounter)], filteredPaths['Gene' + str(geneCounter)])

        # 9. Optimize Transcripts
        m = gp.Model()
    
        # define data coefficients
    
        features = exonSpliceJunctionCountDict['Gene' + str(geneCounter)]
        featureList = list(features.values())
        featureQuantity = len(featureList)
        transcripts = list(filteredPaths['Gene' + str(geneCounter)].values())

        # add decision variables
        frequency = m.addVars(len(transcripts), vtype=gp.GRB.CONTINUOUS, name='frequency')
        frequencySum = m.addVars(len(transcripts), vtype=gp.GRB.CONTINUOUS, name='frequencySum')
        coverageNorms = m.addVars(featureQuantity)
        

        #formula =gp.quicksum(math.sqrt(math.pow(featureList[j] - gp.quicksum(frequency[i]*alphaNP[j][i] for i in range(len(frequency))),2)) for j in range(featureQuantity))
        for j in range(featureQuantity):
            frequencySum[j] = gp.quicksum(frequency[i]*alphaNP[j][i] for i in range(len(frequency)))
        for coverage, featureCount, gamma in zip(coverageNorms.values(), featureList, frequencySum.values()):
            z = m.addVar(lb=-gp.GRB.INFINITY)
            m.addConstr(z==featureCount - gamma)
            m.addGenConstrAbs(coverage, z, 'normconstraint')
        m.setObjective(coverageNorms.sum(), gp.GRB.MINIMIZE)
        m.optimize()
        for v in m.getVars():
            print(v.varName, v.x)

        # 10. Increase geneCounter
        geneCounter = geneCounter + 1

        # All Paths Enumeration

        # Note: source and drain are ALWAYS named "0" and "1" respectively

        #if skip:
            # handle the rare case that noise deletion removes the whole second graph
        

        # TODO WORK WITH THE GRAPH HERE
        #Access Edge Types : G.edges[n1 , n2]['type'] == "Exon" || "SpliceJunction" || "Helper"
        #Access Main Coverage Count of an Edge : G.edges[n1 , n2]['counts']['c']
        #Access Exon length G.edges[n1 , n2]['length']

        #Source Node s is always G.nodes['0']
        #Drain Node t is always G.nodes['1']
        
        # DUMMY Code extracts longest Path (by number of bases) and writes it to a GTF file
        

        lpath = nx.dag_longest_path(G_full, weight="length")
        transcript = nodepath_to_transcript(G_full, lpath)
        write_valid_gtf_entry(dummyf, Chromosome, Strand, Exons, transcript, "Gene"+str(dummyGeneCounter), "Transcript"+str(dummyGeneCounter)+".1")
        dummyGeneCounter = dummyGeneCounter + 1

dummyf.close()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5 rows, 12 columns and 8 nonzeros
Model fingerprint: 0x92a0ab3d
Model has 5 general constraints
Variable types: 12 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 5 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 370 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.700000000000e+02, best bound 3.700000000000e+02, gap 0.0000%
frequency[0] 95.0
frequencySum[0] 0.0
C2 96.0
C3 40.0
C4 146.0
C5 0.0
C6 88.0
C7 96.0
C8 -40.0
C9 146.0
C10 0.0
C11 88.0
{'Gene0': {1: [0, 2]}}
[[0]
 [1]
 [0]
 [1]
 [

In [ ]:
laurasFileEndReached = False
laurasFileName = 'vergleich_postfilter.txt'
laurasTranscriptList = []
with open (laurasFileName, 'r') as laurasFile:
    line = laurasFile.readline()
    while (laurasFileEndReached==False):
        laurasTranscriptList.append(int(line))
        line = laurasFile.readline()
        if line == '':
            laurasFileEndReached = True

print (len(laurasTranscriptList))
print (len(filteredPaths.values()))
richisTranscriptList = list(filteredPaths.values())
erroneousTranscripts = []
summe=0

for i in range(0,len(laurasTranscriptList)):
    if len(richisTranscriptList[i])!=laurasTranscriptList[i]:
        erroneousTranscripts.append(i)
        summe = summe + (laurasTranscriptList[i]-len(richisTranscriptList[i]))
print(erroneousTranscripts)

In [ ]:
print(laurasTranscriptList[694])
print(len(richisTranscriptList[694]))
print(paths['Gene694'])
print(len(paths['Gene694']))
print(filteredPaths['Gene694'])
print(len(filteredPaths['Gene694']))
print(summe)

In [2]:
from printGraph import writeGeneToGraph
writeGeneToGraph(1,'human_geuvadis_simulated_5sets.graph')

Ende vom Graphen noch nicht erreicht.
==META

Ende vom Graphen noch nicht erreicht.
GL000194.1 -

Ende vom Graphen noch nicht erreicht.
Exon  0 53590 53862

Ende vom Graphen noch nicht erreicht.
Exon  1 53863 54677

Ende vom Graphen noch nicht erreicht.
Exon  2 54678 54832

Ende vom Graphen noch nicht erreicht.
Exon  3 54833 55310

Ende vom Graphen noch nicht erreicht.
Exon  4 55430 55445

Ende vom Graphen noch nicht erreicht.
Exon  5 55446 55676

Ende vom Graphen noch nicht erreicht.
Exon  6 56359 56388

Ende vom Graphen noch nicht erreicht.
Exon  7 112792 112850

Ende vom Graphen noch nicht erreicht.
Exon  8 114986 115051

Ende vom Graphen noch nicht erreicht.
==BINS

Ende vom Graphen noch nicht erreicht.
Bin 0,1 Count 4:28;3:46;0:34;2:33;1:33;c:174;

Ende vom Graphen noch nicht erreicht.
Bin 0,2 Count 3:2;1:1;c:3;

Ende vom Graphen noch nicht erreicht.
Bin 0 Count 2:49;4:39;0:27;1:42;3:47;c:204;

Ende vom Graphen noch nicht erreicht.
Bin 1,2 Count 0:29;1:20;3:54;2:36;4:24;c:163;

En